In [ ]:
# -------------------------------------------
# fcc_cat_dog.ipynb - Cats vs Dogs Classifier
# -------------------------------------------

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os, shutil, re
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------------------
# 1. Download and unzip dataset
# -------------------------------------------
!wget https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip
!unzip -q cats_and_dogs.zip

PATH = 'cats_and_dogs'
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

# Move test images into subfolder for flow_from_directory
new_test_subfolder = os.path.join(test_dir, 'images')
os.makedirs(new_test_subfolder, exist_ok=True)

for filename in os.listdir(test_dir):
    if filename.endswith('.jpg'):
        shutil.move(
            os.path.join(test_dir, filename),
            os.path.join(new_test_subfolder, filename)
        )

# -------------------------------------------
# 2. Set constants
# -------------------------------------------
batch_size = 128
epochs = 10
IMG_HEIGHT = 150
IMG_WIDTH = 150

# -------------------------------------------
# 3. Data generators
# -------------------------------------------
image_generator = ImageDataGenerator(rescale=1./255)
train_image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data_gen = train_image_generator.flow_from_directory(
    train_dir, target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size, class_mode='binary'
)

val_data_gen = image_generator.flow_from_directory(
    validation_dir, target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size, class_mode='binary'
)

test_data_gen = image_generator.flow_from_directory(
    test_dir, target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1, class_mode=None, shuffle=False
)

# Sort test filenames numerically
test_data_gen.filenames.sort(key=lambda x: int(re.search(r'\d+', x).group()))

# -------------------------------------------
# 4. Plot function
# -------------------------------------------
def plotImages(images_arr, probabilities=False):
    fig, axes = plt.subplots(1, len(images_arr), figsize=(15, 13))
    for i, ax in enumerate(axes):
        ax.imshow(images_arr[i])
        ax.axis('off')
        if probabilities is not False:
            prob = probabilities[i]
            label = "%.2f%% dog" % (prob * 100) if prob > 0.5 else "%.2f%% cat" % ((1 - prob) * 100)
            ax.set_title(label)
    plt.show()

# -------------------------------------------
# 5. Preview data
# -------------------------------------------
sample_training_images, _ = next(train_data_gen)
plotImages(sample_training_images[:5])

augmented_images = [train_data_gen[0][0][0] for _ in range(5)]
plotImages(augmented_images)

# -------------------------------------------
# 6. Model
# -------------------------------------------
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# -------------------------------------------
# 7. Train model
# -------------------------------------------
history = model.fit(
    train_data_gen,
    steps_per_epoch=62,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=31
)

# -------------------------------------------
# 8. Plot training results
# -------------------------------------------
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))
plt.figure(figsize=(8, 8))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Loss')

plt.show()

# -------------------------------------------
# 9. Prepare test images
# -------------------------------------------
test_data_gen.reset()
test_images = [test_data_gen[i][0] for i in range(len(test_data_gen))]
test_images = np.array(test_images).reshape(-1, 150, 150, 3)

# -------------------------------------------
# 10. Predict
# -------------------------------------------
probabilities = model.predict(test_images)
flat_probs = [p[0] for p in probabilities]
plotImages(test_images[:5], flat_probs[:5])

# -------------------------------------------
# 11. Ground truth comparison
# -------------------------------------------
answers = [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
           1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
           1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
           1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
           0, 0, 0, 0, 0, 0]

correct = sum([round(p) == a for p, a in zip(flat_probs, answers)])
percentage = (correct / len(answers)) * 100

print(f"\n✅ Your model correctly identified {round(percentage, 2)}% of the images of cats and dogs.")
if percentage >= 63:
    print("🎉 You passed the challenge!")
else:
    print("❌ You haven't passed yet. Try again!")
